# Quantum Query Algorithms

### Query Model of Computation

The entire input is not used for the computation
- The input is passed as a *function* and the computation uses the input as a query for answers

- The query models version of computation suggests that inputs only result in answers, you dont get to see the bigger picture of how everything works
    - this is why terms like oracle or black box are used

The input to the query functions in the lesson will take the form

f: ∑^n -> ∑^m
- f denotes a function
- ∑ = {0,1}
- n and m are integers
basically Were saying that we pass in a integer string of length n and recieving another string of length m

a computation makes a query when the n string is selected and the m string is available to the computation

the efficiency of an algorithm is calculated by counting the amount of queries the input requires

### Examples of Query Problems

#### Or (me lol)
input : ∑^n -> ∑ (returns a one digit string from the input)
output: 1 if theres a 1 in the string, 0 if no ones in the string

#### Minimum
input: f: ∑^n -> ∑^m
output: the minimum value or more specifically, the string that comes fist when read lexicographically (first one in the string)


**Promise** - The input is going to have to exist some sort of way to get useful computation, the computation wont evaluate the string seriously if its not in a certain way

#### Unique Search
input : ∑^n -> ∑ (returns a one digit string from the input)
output: returns a string z for the unique item searched

Query problems usually dont feel natural or intuitive like this, sometimes the input and the querying dont feel natural

### Query Gates

- in classical boolean circuits, we can imagine the input as an the function of n length of bits, and the circuit as the computation

![title](01A.png)

think of the circuits as values that never change and the function as the variable that changes

Query gates are the inputs, we are given the input, and we are expected to build the computation for the output

### Quantum Circuit Model

Since Quantum states can be in superposition, there will be situations where the quantum state for a circuit will be in a non-unitary state. The function gate has to be changed into a unitary query gate that can operate on a standard basis state

![title](02A.png)

The above example is a bitwase exclusive Or (XOR) function. It returns a 1 for any 2 numbewers bewteen |X> and |Y> where one is a one and the other is a 0. 

The matrix is an example of a permutation matrix. It changes the order of the string, thats it
This just measn its an identity matrix with some rows/columns flipped around, ergo its unitary

NOTE: it is realtively straightforward to create a quantum analogue of a classical query circuit

### Deutchs Algorithm

Deutchs Problem - parity problem for functions of the form F: ∑ -> ∑ (one bit to one bit)
function takes single bit as input, returns 0 if f is constant f(01)= (00),(11) or 1 if f is balanced f(01)=(01),(10)

every classical query algorithm must make 2 queries to f to solve this problem.
- you need to know both bits to find the parity

Deutschs algorithm can solve the parity problem with one query to a quantum circuit

![title](03A.png)
Init qubit 0 to |0> and qubit 1 to |1>
Perform Hadamard ops. on both of the qubits, feed them into the query gate U, perform one more Hadamard, then measure

measuring a zero means its constant, measuring a 1 means its balanced

After the Hadamard on both qubits, we are in the tensor product of the - and + states
| - >|+ > == 1/2(|0> -|1>)|0> + 1/2(|0> - |1>)|1>

Next, the Unitary matrix query gate XOR's the top qubit into the bottom qubit giving the state

1/2 (∣0⊕f(0)⟩−∣1⊕f(0)⟩)∣0⟩+  1/2 (∣0⊕f(1)⟩−∣1⊕f(1)⟩)∣1⟩

when we perform the final hadamard op. we get 

-1 ^(f(0)) |-> (|0> or |1>) 

why this algorithm works is that when we place the Hadamard on the first qubit, then we are in a superposition, then when we compute the unitary matrix, we use the interference of the second qubit to guide us to the right answer. The phase of the second qubit interferes, bringing a higher likelihood to the right answer and a lower likelihood for the wrong answer

#### Phase Kickback
(the phase of one qubit is affected by the state of another qubit)
|b ® c> = X^c |b>

b and c are random binary values
XORing a bit b by the value 1 gives you the inverse (or the NOT gate) of b

performing the Deutsch gate on the standard basis states |b>|a> gives you the XOR of b and the function of a tensored with |a>
|b ® f(a)>|a>

the matrix X has eigenvalue -1, meaning that X|-> = - | ->

In [1]:
from qiskit import QuantumCircuit


def deutsch_function(case: int):
    """
    Generate a valid Deutsch function as a `QuantumCircuit`.
    """
    if case not in [1, 2, 3, 4]:
        raise ValueError("`case` must be 1, 2, 3, or 4.")

    f = QuantumCircuit(2)
    if case in [2, 3]:
        f.cx(0, 1)
    if case in [3, 4]:
        f.x(1)
    return f

In [2]:
deutsch_function(3).draw()

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├
     └───┘└───┘

In [3]:
# Now the Unitary operation can be applied

def compile_circuit(function: QuantumCircuit):
    """
    Compiles a circuit for use in Deutsch's algorithm.
    """
    n = function.num_qubits - 1
    qc = QuantumCircuit(n + 1, n)

    qc.x(n)
    qc.h(range(n + 1))

    qc.barrier()
    qc.compose(function, inplace=True)
    qc.barrier()

    qc.h(range(n))
    qc.measure(range(n), range(n))

    return qc

In [4]:
compile_circuit(
    deutsch_function(3)
).draw()

┌───┐      ░            ░ ┌───┐┌─┐
q_0: ┤ H ├──────░───■────────░─┤ H ├┤M├
     ├───┤┌───┐ ░ ┌─┴─┐┌───┐ ░ └───┘└╥┘
q_1: ┤ X ├┤ H ├─░─┤ X ├┤ X ├─░───────╫─
     └───┘└───┘ ░ └───┘└───┘ ░       ║ 
c: 1/════════════════════════════════╩═
                                     0

### Deutsch- Josza algorithm

Basically Deutsch algorithm but goes from one bit to one bit, to n bits to one bit
f: ∑^n -> ∑

Deutch-Josza Circuit
![title](04A.png)
the output is a n-bit string y that tells us something about the function f

Deutsch-Josza Problem:
tells us if the input neither constant or balanced (0 , 1 respectively)
We promise that the input functions are either constant or balanced (some n>=2 strings can be neither, these are the 'dont care' values )

#### Analysis

The Hadamard operation can be rewritten for these standard basis states 

![title](05A.png)

A Hadamard for a layer of qubits can be expressed like this 

![title](06A.png)

the tensor product in the exponent means that we are tensor producting H with itself n times

##### Binary Dot Product
- the XOR of the AND of the bits of each string
- 1 if the sum of each AND is odd
- 0 if the sum of each AND is even



After the circuit, we have a constant bit-string if every qubit measures to 0
Or its balanced if one of the qubit measures to 1

Any deterministic algorithm needs to make 2^(n-1)+1 queries (more than half of n)
A probabalistic algorithm can be pretty accurate (not perfect) by randomly choosing bits in the string and seeing if they have different values or the same values


### The Berenstein-Vazirani Problem
input f: ∑^n -> ∑
promise: Theres a string S wfor which f(x) = s*x (binary dot product)
output: S

The Deutsch-Josza Algorithm Can give the answer with no change to its circuit

![title](07A.png)

which can be reduced to 
| - > ® | s >

### Simon's Algorithm

input: F: ∑^n -> ∑^m (maps a string of length n to a string of length m)
promise: f(x)=f(y) <=> (x=y) or (x ® s = y)
output: S

Cases for S
S= 0^n (zero string)
- Results in f(x) = f(y) being x=y because any strin XOR'd with itself is itself (one-to-one)

S != o^n
- Results in f(x) = f(x ® s)(two-to-one)
    - for every inout string x, there must be exactly one other string where f(x)= f(x ® s)

Case 2 table
![title](08A.png)

we can see that f(000) == f(011) = 10011, f(001) = f(010) = 00101,...

The Circuit for simons algorithm runs the circuit several times, and has m qubits in the beginning, but returns n qubits

![title](09A.png)

n qubits go through an H. operation, m qubits stay initialized at |0>, meaning there is no phase kickback in the algorithm

each time we run the circuit, we get some info in the form of a string y that says something about the string S
running it multiple times increases the confidence in what S might be

### Classical Post-Processing
Running the circuit gives us a random string y 
suppose we run the string k time (k=n+r) giving us k strings y - we will have probability 99.9%
